In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
ser = Service('C:/Users/jjp75/Downloads/chromedriver.exe') # chromedriver.exe의 절대적 위치
driver = webdriver.Chrome(service = ser)                            # driver 객체 생성
url = "https://www.starbucks.co.kr/store/store_map.do/"    #  크롤링을 실행 할 url 주소 - 스타벅스 매장위치 사이트
driver.get(url)                                                                    # chromedriver을 통해서 url에 접근

In [91]:
# [지역검색] 버튼의 SELECTOR 값
loc_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
#  [서울] 버튼의 SELECTOR 값
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
# [전체] 버튼의 SELECTOR 값
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
# [지역검색] 버튼을 클릭
driver.find_element(By.CSS_SELECTOR, loc_btn).click()
# time.sleep이 없이 진행하면 명령어들이 거의 동시에 진행되어 웹 상에서 반응 하기 전에 끝나버렸다. 
#클릭후 페이지 로드 시간을 감안하여 1초씩 실행시간을 부여했다.
time.sleep(1) 
# [서울] 버튼을 클릭
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()
time.sleep(1)
# [전체] 버튼을 클릭
driver.find_element(By.CSS_SELECTOR, all_btn).click()

In [84]:
html = driver.page_source                         # 서울 전체 매장 정보가 나타나있는 페이지의 소스를 html 변수에 지정
soup = BeautifulSoup(html, 'html.parser')  # BeautifulSoup을 활용하여 html parse

In [173]:
# 매장 리스트의 SELECTOR 값을 통해 모든 매장 정보를 저장
store_list = soup.select('#mCSB_3_container > ul > li.quickResultLstCon') 
# store_list의 길이를 출력해본 결과 577개로 서울 내 모든 매장정보다 담겼음을 알 수 있다.
len(store_list)

577

In [184]:
# for 문을 돌려가며 store_list에서 데이터를 뽑아오기 전 데이터 형태 확인하는 작업
test = store_list[0]
test.select('p')[0].text

'서울특별시 강남구 언주로 425 (역삼동)1522-3232'

In [171]:
df = pd.DataFrame() # 빈 데이터프레임 생성, 여기에 store_list에서 뽑인 정보가 들어갈 것이다.
for store in store_list:
    # store_list의 개별 store에서 'data-lat' 'data-long' attribute를 통해 위도 경도 정보를 얻는다.
    ## bs4.element 객체에서 attribute의 value에 접근하기 위해서는 element['attribute'] <- 이런식을 통해 value값을 얻는다.
    lat_long = (store['data-lat'],store['data-long']) 
    # 'data-name' attribute에 지정되어있는 매장이름 정보
    name = store['data-name']
    # ㅇㅇ시 ㅇㅇ구 형태로 되어있는 부분을 공백을 기준으로 split하여 adr_list 변수에 list형태로 저장했다.
    adr_list = store.select('p')[0].text.split(" ")
    # adr_list의 시 정보에 대한 데이터 부분이다. '서울특별시' 라고 되어있어 앞의 두글자만 따와서 '서울'만 저장했다.
    si = adr_list[0][:2]
    # adr_list의 구 정보
    gu = adr_list[1]
    # 동 정보가 있는경우 (ㅇㅇ동) 형태로 표기되어있었는데 동 정보가 없는 경우도 있었다. 
    # 그런 경우엔 except문을 통해 도로명 주소가 나타나도록 했다.
    try:
        dong = store.select('p')[0].text.split("(")[1].split(")")[0]
    except:
        dong = adr_list[2]
    # 위에서 모은 시 구 동 위경도 매장명 데이터를 Series 형태로 저장
    x = pd.Series([si,gu,dong,lat_long,name])
    # 데이터프레임에 하나씩 추가
    df = df.append(x, ignore_index = True)

In [178]:
# 저장된 csv 파일을 확인 결과 잘 저장이 되었음을 확인할 수 있었다.
df.to_csv("store_list.csv")